In [1]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import pandas as pd
import tensorflow as tf
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation, Input, Reshape, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import copy
from tensorflow.keras.regularizers import l2
from tensorflow.keras import utils
import pymongo
import ta
import numpy as np

Init Plugin
Init Graph Optimizer
Init Kernel


/var/folders/qg/l38qyqxd7nj0gg6p22rjgd600000gn/T/ipykernel_10715/4230669043.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [42]:
def get_nn_input(data, days):
    test_num = 2*20*8
    dont_norm = ['trend_psar_down_indicator','trend_psar_up_indicator','volatility_kchi','volatility_bbli','volatility_bbhi','volatility_kcli']
    tmp = copy.deepcopy(data['Close'].to_numpy())
    goal = []
    for i in range(len(tmp)-2):
        goal.append( (tmp[i+1] - tmp[i]) / tmp[i] * 100)
    goal.append(0)
    goal.append(0)
    goal = np.array(goal)
    for i in data.columns[:-4]:
        if i not in dont_norm:
            data[i] = (data[i] - data[i].rolling(days*24).mean())/data[i].rolling(days*24).std()
    data = data.fillna(0)
    data = data[data.columns[:-4]]
    data = data.to_numpy()
    goal = goal[10:-10]
    data = data[10:-10]
    xtest = data[-test_num:]
    ytest = goal[-test_num:]
    data = data[:-test_num]
    goal = goal[:-test_num]
    return data, goal, xtest, ytest

In [43]:
def get_indicators(data):
    data = ta.add_all_ta_features(
            data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
    return data

In [104]:
import os 
files = ['MSFT','AAPL','AMZN','TSLA','GOOGL','GOOG','FB','NVDA','JPM','HD','JNJ','PG','UNH','BAC','V','ADBE','NFLX','PFE','DIS']
print(len(files))
c = 0
X = None
y = None
Xtest = None
ytest = None
for file in files[:30]:
    c += 1
    if c % 100 == 0:
        print(c)
    path = './data/'
    data = pd.read_csv(path+file + '.CSV')
    try:
        data = data.drop(['Adj Close','Unnamed: 0'],axis=1)
    except:
        continue
    try:
        data = get_indicators(data)
        
    except:
        continue
    if X is None:
        X,y, Xtest, ytest = get_nn_input(data, 30)
        X = X[31*24:]
        y = y[31*24:]
    else:
        Xtmp, ytmp, Xtesttmp, ytesttmp = get_nn_input(data, 30)
        Xtmp = Xtmp[31*24:]
        ytmp = ytmp[31*24:]
        X = np.append(X, Xtmp, axis = 0)
        y = np.append(y, ytmp)
        Xtest = np.append(Xtest, Xtesttmp, axis = 0)
        ytest = np.append(ytest, ytesttmp)

19


In [105]:
yclass = []
probs = [0,0]
for i in y:
    if i > 0:
        yclass.append([0,1])
        probs[1] += 1
    else:
        yclass.append([1,0])
        probs[0] += 1
ytestclass = []
for i in ytest:
    if i > 0:
        ytestclass.append([0,1])
    else:
        ytestclass.append([1,0])
yclass = np.array(yclass)
ytestclass = np.array(ytestclass)

print(probs)

[18685, 19372]


In [106]:
from sklearn.model_selection import train_test_split

X, Xval, yclass, yclassval = train_test_split(X, yclass, test_size = 0.25, shuffle= True)

In [114]:
def simple_class():
    input_lmd = Input(shape=(84))
    out = Dropout(0.2)(input_lmd)
    out = Dense(25, activation='relu')(out)
    out = Dropout(0.5)(out)
    out = Dense(100, activation='relu')(out)
#     out = Dropout(0.4)(out)\
#     out = Dense(100, activation='relu')(out)
#     out = Dropout(0.5)(out)
    out = Dense(2, activation='softmax')(out)
    model = Model(inputs=input_lmd, outputs=out)
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model
model = simple_class()

Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 84)]              0         
_________________________________________________________________
dropout_46 (Dropout)         (None, 84)                0         
_________________________________________________________________
dense_62 (Dense)             (None, 25)                2125      
_________________________________________________________________
dropout_47 (Dropout)         (None, 25)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 100)               2600      
_________________________________________________________________
dense_64 (Dense)             (None, 2)                 202       
Total params: 4,927
Trainable params: 4,927
Non-trainable params: 0
________________________________________________________

In [115]:
early = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=20, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

In [116]:
model.fit(X,yclass,64,800,validation_data=(Xval,yclassval),use_multiprocessing=True,verbose=1,callbacks=[early])

Epoch 1/800
  6/446 [..............................] - ETA: 4s - loss: 0.7710 - accuracy: 0.4896  

2021-11-24 13:06:52.282012: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


446/446 [==============================] - ETA: 0s - loss: 0.7167 - accuracy: 0.5039

2021-11-24 13:06:55.502423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


446/446 [==============================] - 4s 8ms/step - loss: 0.7167 - accuracy: 0.5039 - val_loss: 0.6960 - val_accuracy: 0.4987
Epoch 2/800
446/446 [==============================] - 3s 8ms/step - loss: 0.7022 - accuracy: 0.5018 - val_loss: 0.6947 - val_accuracy: 0.5005
Epoch 3/800
446/446 [==============================] - 4s 8ms/step - loss: 0.6999 - accuracy: 0.4990 - val_loss: 0.6944 - val_accuracy: 0.4972
Epoch 4/800
446/446 [==============================] - 4s 8ms/step - loss: 0.6978 - accuracy: 0.5006 - val_loss: 0.6951 - val_accuracy: 0.4988
Epoch 5/800
446/446 [==============================] - 4s 8ms/step - loss: 0.6971 - accuracy: 0.5000 - val_loss: 0.6937 - val_accuracy: 0.5042
Epoch 6/800
446/446 [==============================] - 4s 8ms/step - loss: 0.6961 - accuracy: 0.5017 - val_loss: 0.6939 - val_accuracy: 0.5013
Epoch 7/800
446/446 [==============================] - 4s 8ms/step - loss: 0.6957 - accuracy: 0.5033 - val_loss: 0.6938 - val_accuracy: 0.5032
Epoch 8/800

In [117]:
preds = model.predict(Xtest)
preds = np.array(preds).squeeze()
yclasstest = np.array(ytest).squeeze()
print(len(preds),len(yclasstest))
print(preds)

2021-11-24 13:11:54.964972: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


6080 6080
[[0.4823471  0.5176529 ]
 [0.49064395 0.5093561 ]
 [0.48295707 0.517043  ]
 ...
 [0.5471311  0.45286888]
 [0.52592    0.47408006]
 [0.5280496  0.47195047]]


In [123]:
def class_trade_env(preds, real, a):
    balance = 100
    num = 0
    for i in range(len(preds)):
        if preds[i][1] > 0.7:
            balance *= 1 + real[i]/100 
            num += 1
        if preds[i][0] >  0.7:
            balance *= 1 - real[i]/100 
            num += 1
    return balance, num
class_trade_env(preds,yclasstest,0)

(100, 0)

In [37]:
import warnings
warnings.filterwarnings('ignore')
rev = {}
files = ['dash.csv','zec.csv','bitcoin.csv', 'ethereum.csv','bch.csv','eos.csv','neo.csv','bnb.csv','etc.csv']#,'zec.csv']

for file in files:
    symbol = file.split('.')[0]
    path = './train/'
    data = pd.read_csv(path+file,header = None)
    data = data.drop([6,7,9,10,11,12],axis=1)
    data.columns = ['index','time','open','high','low','close','volume']
    data = get_indicators(data)
    
    X,y, Xtest, ytest = get_nn_input(data, 30)
    X = X[90*24:]
    y = y[90*24:]
    bound = 0.25 * np.std(y)
    print(bound)
    yclass = []
    probs = [0,0,0]
    for i in y:
        if i > bound:
            yclass.append([0,0,1])
            probs[2] += 1
        elif i < -bound:
            yclass.append([1,0,0])
            probs[0] += 1
        else:
            yclass.append([0,1,0])
            probs[1] += 1
    ytestclass = []
    for i in ytest:
        if i > bound:
            ytestclass.append([0,0,1])
        elif i < -bound:
            ytestclass.append([1,0,0])
        else:
            ytestclass.append([0,1,0])
    yclass = np.array(yclass)
    ytestclass = np.array(ytestclass)
    print(probs)
    from sklearn.model_selection import train_test_split

    X, Xval, yclass, yclassval = train_test_split(X, yclass, test_size = 0.25, shuffle= True)
    model = simple_class()
    model.fit(X,yclass,128,2000,validation_data=(Xval,yclassval),use_multiprocessing=True,verbose=2,callbacks=[early])
    preds = model.predict(Xtest)
    preds = np.array(preds).squeeze()
    yclasstest = np.array(ytest).squeeze()
    print(len(preds),len(yclasstest))
    maxi = 100
    kret = 0
    for k in range(10,20):
        k /= 10
        tmp =  class_trade_env(preds,yclasstest,k)
        if tmp[0] > maxi:
            maxi = tmp[0]
            kret = k
    model.save('./models/'+ symbol + '.h5')
    rev[symbol] = {'k':kret, 'ret':maxi, 'bound':bound}
    print(rev[symbol],symbol)


0.33702117754901834
[5745, 6859, 5777]
Epoch 1/2000


2021-11-23 15:17:07.740201: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-23 15:17:08.596888: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


108/108 - 1s - loss: 1.1778 - accuracy: 0.3532 - val_loss: 1.0843 - val_accuracy: 0.4175
Epoch 2/2000
108/108 - 1s - loss: 1.1044 - accuracy: 0.3721 - val_loss: 1.0828 - val_accuracy: 0.4117
Epoch 3/2000
108/108 - 1s - loss: 1.0952 - accuracy: 0.3784 - val_loss: 1.0811 - val_accuracy: 0.4193
Epoch 4/2000
108/108 - 1s - loss: 1.0907 - accuracy: 0.3942 - val_loss: 1.0802 - val_accuracy: 0.4197
Epoch 5/2000
108/108 - 1s - loss: 1.0857 - accuracy: 0.4018 - val_loss: 1.0774 - val_accuracy: 0.4310
Epoch 6/2000
108/108 - 1s - loss: 1.0845 - accuracy: 0.4064 - val_loss: 1.0767 - val_accuracy: 0.4295
Epoch 7/2000
108/108 - 1s - loss: 1.0823 - accuracy: 0.4076 - val_loss: 1.0743 - val_accuracy: 0.4299
Epoch 8/2000
108/108 - 1s - loss: 1.0800 - accuracy: 0.4094 - val_loss: 1.0745 - val_accuracy: 0.4282
Epoch 9/2000
108/108 - 1s - loss: 1.0806 - accuracy: 0.4065 - val_loss: 1.0756 - val_accuracy: 0.4247
Epoch 10/2000
108/108 - 1s - loss: 1.0774 - accuracy: 0.4100 - val_loss: 1.0742 - val_accuracy:

Epoch 81/2000
108/108 - 1s - loss: 1.0490 - accuracy: 0.4442 - val_loss: 1.0629 - val_accuracy: 0.4386
Epoch 82/2000
108/108 - 1s - loss: 1.0487 - accuracy: 0.4442 - val_loss: 1.0629 - val_accuracy: 0.4399
Epoch 83/2000
108/108 - 1s - loss: 1.0473 - accuracy: 0.4461 - val_loss: 1.0625 - val_accuracy: 0.4421
Epoch 84/2000


KeyboardInterrupt: 